In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf



In [54]:
# Read the JSON file into a Pandas DataFrame
df = pd.read_json('Loan_approval_dataset.json')

df.head()

,Id,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,1,1303834,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


In [55]:
df.describe()


,Id,Income,Age,Experience,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
count,252000.000000,2.520000e+05,252000.000000,252000.000000,252000.000000,252000.000000,252000.000000
mean,126000.500000,4.997117e+06,49.954071,10.084437,6.333877,11.997794,0.123000
std,72746.278255,2.878311e+06,17.063855,6.002590,3.647053,1.399037,0.328438
min,1.000000,1.031000e+04,21.000000,0.000000,0.000000,10.000000,0.000000
25%,63000.750000,2.503015e+06,35.000000,5.000000,3.000000,11.000000,0.000000
50%,126000.500000,5.000694e+06,50.000000,10.000000,6.000000,12.000000,0.000000
75%,189000.250000,7.477502e+06,65.000000,15.000000,9.000000,13.000000,0.000000
max,252000.000000,9.999938e+06,79.000000,20.000000,14.000000,14.000000,1.000000


In [56]:
df.dtypes

Id                    int64
Income                int64
Age                   int64
Experience            int64
Married/Single       object
House_Ownership      object
Car_Ownership        object
Profession           object
CITY                 object
STATE                object
CURRENT_JOB_YRS       int64
CURRENT_HOUSE_YRS     int64
Risk_Flag             int64
dtype: object

In [57]:
unique_states_count = len(df["STATE"].unique())
print(unique_states_count)

29


In [58]:
state_app_counts = df['STATE'].value_counts()
state_app_counts

STATE
Uttar_Pradesh        28400
Maharashtra          25562
Andhra_Pradesh       25297
West_Bengal          23483
Bihar                19780
Tamil_Nadu           16537
Madhya_Pradesh       14122
Karnataka            11855
Gujarat              11408
Rajasthan             9174
Jharkhand             8965
Haryana               7890
Telangana             7524
Assam                 7062
Kerala                5805
Delhi                 5490
Punjab                4720
Odisha                4658
Chhattisgarh          3834
Uttarakhand           1874
Jammu_and_Kashmir     1780
Puducherry            1433
Mizoram                849
Manipur                849
Himachal_Pradesh       833
Tripura                809
Uttar_Pradesh[5]       743
Chandigarh             656
Sikkim                 608
Name: count, dtype: int64

In [59]:
# place some states (<2000 applications) into an 'OTHER' column to reduce dimensionality
states_to_replace = state_app_counts[state_app_counts < 2000].index.tolist()

for state in states_to_replace:
    df['STATE'] = df['STATE'].replace(state, "Other")

df['STATE'].value_counts()

STATE
Uttar_Pradesh     28400
Maharashtra       25562
Andhra_Pradesh    25297
West_Bengal       23483
Bihar             19780
Tamil_Nadu        16537
Madhya_Pradesh    14122
Karnataka         11855
Gujarat           11408
Other             10434
Rajasthan          9174
Jharkhand          8965
Haryana            7890
Telangana          7524
Assam              7062
Kerala             5805
Delhi              5490
Punjab             4720
Odisha             4658
Chhattisgarh       3834
Name: count, dtype: int64

In [60]:
unique_cities_count = len(df["CITY"].unique())
print(unique_cities_count)

317


In [61]:
# Drop the cities column - see if using just the states as geographic area works: to reduce dimensionality
df = df.drop("CITY", axis=1)

In [62]:
df['Profession'].value_counts()

Profession
Physician                     5957
Statistician                  5806
Web_designer                  5397
Psychologist                  5390
Computer_hardware_engineer    5372
Drafter                       5359
Magistrate                    5357
Fashion_Designer              5304
Air_traffic_controller        5281
Comedian                      5259
Industrial_Engineer           5250
Mechanical_engineer           5217
Chemical_engineer             5205
Technical_writer              5195
Hotel_Manager                 5178
Financial_Analyst             5167
Graphic_Designer              5166
Flight_attendant              5128
Biomedical_Engineer           5127
Secretary                     5061
Software_Developer            5053
Petroleum_Engineer            5041
Police_officer                5035
Computer_operator             4990
Politician                    4944
Microbiologist                4881
Technician                    4864
Artist                        4861
Lawyer   

In [63]:
# reindex the data frame to set the Id as the index
df.set_index('Id', inplace=True)

In [64]:
# loop through values in Married/Single column to make a new column with 0 for married, 1 for single
df = pd.get_dummies(df, columns=['Married/Single'],dtype= int)

df.head()

,Income,Age,Experience,House_Ownership,Car_Ownership,Profession,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag,Married/Single_married,Married/Single_single
Id,,,,,,,,,,,,
1,1303834,23,3,rented,no,Mechanical_engineer,Madhya_Pradesh,3,13,0,0,1
2,7574516,40,10,rented,no,Software_Developer,Maharashtra,9,13,0,0,1
3,3991815,66,4,rented,no,Technical_writer,Kerala,4,10,0,1,0
4,6256451,41,2,rented,yes,Software_Developer,Odisha,2,12,1,0,1
5,5768871,47,11,rented,no,Civil_servant,Tamil_Nadu,3,14,1,0,1


In [65]:
# Encode Ownership columns with get_dummies

df= pd.get_dummies(df, columns=['Car_Ownership'],dtype= int)
df=pd.get_dummies(df,columns=['House_Ownership'],dtype= int)
df.head()

,Income,Age,Experience,Profession,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag,Married/Single_married,Married/Single_single,Car_Ownership_no,Car_Ownership_yes,House_Ownership_norent_noown,House_Ownership_owned,House_Ownership_rented
Id,,,,,,,,,,,,,,,
1,1303834,23,3,Mechanical_engineer,Madhya_Pradesh,3,13,0,0,1,1,0,0,0,1
2,7574516,40,10,Software_Developer,Maharashtra,9,13,0,0,1,1,0,0,0,1
3,3991815,66,4,Technical_writer,Kerala,4,10,0,1,0,1,0,0,0,1
4,6256451,41,2,Software_Developer,Odisha,2,12,1,0,1,0,1,0,0,1
5,5768871,47,11,Civil_servant,Tamil_Nadu,3,14,1,0,1,1,0,0,0,1


In [66]:
# drop the married column  and the Car_Ownership no column to reduce the dimensionality, but retain binary nature of this information
df = df.drop("Married/Single_married", axis=1)
df = df.drop("Car_Ownership_no", axis=1)
df.head()

,Income,Age,Experience,Profession,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag,Married/Single_single,Car_Ownership_yes,House_Ownership_norent_noown,House_Ownership_owned,House_Ownership_rented
Id,,,,,,,,,,,,,
1,1303834,23,3,Mechanical_engineer,Madhya_Pradesh,3,13,0,1,0,0,0,1
2,7574516,40,10,Software_Developer,Maharashtra,9,13,0,1,0,0,0,1
3,3991815,66,4,Technical_writer,Kerala,4,10,0,0,0,0,0,1
4,6256451,41,2,Software_Developer,Odisha,2,12,1,1,1,0,0,1
5,5768871,47,11,Civil_servant,Tamil_Nadu,3,14,1,1,0,0,0,1


In [67]:
# Encode the profession column with get_dummies
df= pd.get_dummies(df, columns=['Profession'],dtype= int)

In [68]:
# Encode the State column with git_dummies
df=pd.get_dummies(df, columns=['STATE'], dtype=int)
df.head()

,Income,Age,Experience,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag,Married/Single_single,Car_Ownership_yes,House_Ownership_norent_noown,House_Ownership_owned,...,STATE_Madhya_Pradesh,STATE_Maharashtra,STATE_Odisha,STATE_Other,STATE_Punjab,STATE_Rajasthan,STATE_Tamil_Nadu,STATE_Telangana,STATE_Uttar_Pradesh,STATE_West_Bengal
Id,,,,,,,,,,,,,,,,,,,,,
1,1303834,23,3,3,13,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,7574516,40,10,9,13,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,3991815,66,4,4,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6256451,41,2,2,12,1,1,1,0,0,...,0,0,1,0,0,0,0,0,0,0
5,5768871,47,11,3,14,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [69]:
# Separate the data into labels and features
# Separate the y variable, the labels
y = df["Risk_Flag"]

# Separate the X variable, the features
X = df.drop("Risk_Flag", axis=1)

In [70]:
df.head()

,Income,Age,Experience,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag,Married/Single_single,Car_Ownership_yes,House_Ownership_norent_noown,House_Ownership_owned,...,STATE_Madhya_Pradesh,STATE_Maharashtra,STATE_Odisha,STATE_Other,STATE_Punjab,STATE_Rajasthan,STATE_Tamil_Nadu,STATE_Telangana,STATE_Uttar_Pradesh,STATE_West_Bengal
Id,,,,,,,,,,,,,,,,,,,,,
1,1303834,23,3,3,13,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,7574516,40,10,9,13,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,3991815,66,4,4,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6256451,41,2,2,12,1,1,1,0,0,...,0,0,1,0,0,0,0,0,0,0
5,5768871,47,11,3,14,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [71]:
# Split the data using train_test_split
# Assign a random_state of 5 to the function
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=5, stratify=y)

In [72]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [73]:
df.to_csv("Loan_approval_for_training.csv")

In [75]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
inputs = 81
layer1_units = 150
layer2_units = 90
layer3_units = 30
layer4_units = 16
layer1_activation = "relu"
layer2_activation = "relu"
layer3_activation = "tanh"
layer4_activation = "tanh"

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = layer1_units, activation = layer1_activation, input_dim = inputs))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = layer2_units, activation = layer2_activation))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = layer3_units, activation = layer2_activation))

#Fourth hidden layer
nn.add(tf.keras.layers.Dense(units = layer4_units, activation = layer4_activation))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_15 (Dense)                │ (None, 150)            │        12,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 90)             │        13,590 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 30)             │         2,730 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 16)             │           496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,133 (113.80 KB)

 Trainable params: 29,133 (113.80 KB)

 Non-trainable params: 0 (0.00 B)

In [76]:
# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer = "adam", metrics=["accuracy"])

import math
batch_size=32
num_batches= len(X_train_scaled)/batch_size
num_batches= math.ceil(num_batches)
num_batches

In [77]:
# Train the model
nn_fit = nn.fit(X_train_scaled, y_train, epochs=25)

Epoch 1/25
5907/5907 ━━━━━━━━━━━━━━━━━━━━ 6s 827us/step - accuracy: 0.8765 - loss: 0.3577
Epoch 2/25
5907/5907 ━━━━━━━━━━━━━━━━━━━━ 5s 853us/step - accuracy: 0.8855 - loss: 0.2893
Epoch 3/25
5907/5907 ━━━━━━━━━━━━━━━━━━━━ 5s 824us/step - accuracy: 0.8901 - loss: 0.2572
Epoch 4/25
5907/5907 ━━━━━━━━━━━━━━━━━━━━ 5s 867us/step - accuracy: 0.8923 - loss: 0.2454
Epoch 5/25
5907/5907 ━━━━━━━━━━━━━━━━━━━━ 5s 839us/step - accuracy: 0.8964 - loss: 0.2324
Epoch 6/25
5907/5907 ━━━━━━━━━━━━━━━━━━━━ 5s 841us/step - accuracy: 0.8981 - loss: 0.2261
Epoch 7/25
5907/5907 ━━━━━━━━━━━━━━━━━━━━ 5s 837us/step - accuracy: 0.8970 - loss: 0.2233
Epoch 8/25
5907/5907 ━━━━━━━━━━━━━━━━━━━━ 5s 846us/step - accuracy: 0.8985 - loss: 0.2162
Epoch 9/25
5907/5907 ━━━━━━━━━━━━━━━━━━━━ 5s 819us/step - accuracy: 0.8992 - loss: 0.2141
Epoch 10/25
5907/5907 ━━━━━━━━━━━━━━━━━━━━ 5s 842us/step - accuracy: 0.8990 - loss: 0.2086
Epoch 11/25
5907/5907 ━━━━━━━━━━━━━━━━━━━━ 5s 832us/step - accuracy: 0.8983 - loss: 0.2076
Epoch 12

In [79]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"loss: {model_loss}, accuracy: {model_accuracy}")

1969/1969 - 1s - 696us/step - accuracy: 0.8894 - loss: 0.2511
loss: 0.25114214420318604, accuracy: 0.8894285559654236
